# Model Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

In [2]:
tf.__version__

'2.5.0'

### Definitions

In [3]:
PATH_TO_SAVE_DATA = "."
PATH_DF_POS_FR = PATH_TO_SAVE_DATA + '/' + 'df_pos_fr.csv'
PATH_DF_TEST_FR = PATH_TO_SAVE_DATA + '/' + 'df_test_fr.csv'
PATH_JSON_METEO_FR = PATH_TO_SAVE_DATA + '/' + 'data_meteo_fr.json'
PATH_DF_FEAT_FR = PATH_TO_SAVE_DATA + '/' + 'df_feat_fr.csv' 
PATH_GEO_DEP_FR = PATH_TO_SAVE_DATA + '/sources/geofrance/' + 'departments.csv'
PATH_MDL_MULTI_STEP = PATH_TO_SAVE_DATA + '/' + "mdl_multi_step_pos_fr_tcn"
PATH_MDL_MULTI_TFLITE = PATH_TO_SAVE_DATA + '/' + \
    'serverless/tensorflow_lite_on_aws_lambda'
PATH_MDL_MULTI_TFLITE_FILE = PATH_MDL_MULTI_TFLITE + '/' + \
    "converted_model.tflite"
PATH_SERVERLESS = PATH_MDL_MULTI_TFLITE + '/' + 'serverless.yml'

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140734 # on 13/05/2020
NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer'

# model 
PAST_HISTORY = 14 # days used to predict next values in future
FUTURE_TARGET = 7 # nb predict days later
STEP = 1

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")

TRAIN_SPLIT = 405


### helper functions

In [4]:
# save file before update
def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
       
        res_re = re.search('\.\w+$', path_file_name)
        
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
         
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))

### load data

In [5]:
df_feat_fr = pd.read_csv(PATH_DF_FEAT_FR)
df_feat_fr.index = df_feat_fr["date"]
df_feat_fr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_fr.shape[0])]
df_feat_fr


,T_min,date,T_max,H_min,H_max,extrap,pos,age_pos,test,age_test,day_num,nb_cases,sum_cases,Rt,rate_pos,train
date,,,,,,,,,,,,,,,,
2020-05-13,284.926667,2020-05-13,290.505000,64.661017,88.135593,0.0,881,61.104427,39013,55.451567,3,141108,NaN,NaN,2.258222,True
2020-05-14,285.050000,2020-05-14,290.963333,59.406780,84.847458,0.0,981,60.403670,41975,54.832186,4,142089,NaN,NaN,2.337105,True
2020-05-15,285.308333,2020-05-15,291.920000,57.372881,82.966102,0.0,1019,60.063788,47596,54.404446,5,143108,NaN,NaN,2.140936,True
2020-05-16,284.956667,2020-05-16,293.500000,53.741379,86.534483,0.0,291,60.020619,16523,54.480058,6,143399,NaN,NaN,1.761181,True
2020-05-17,285.598333,2020-05-17,294.446667,49.879310,85.500000,0.0,141,61.248227,6611,58.226895,0,143540,NaN,NaN,2.132809,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-26,289.327966,2021-09-26,295.911017,63.152542,93.118644,0.0,1114,41.244165,100587,42.809677,0,6914879,86104.0,0.549098,1.107499,False
2021-09-27,287.765000,2021-09-27,295.293333,59.216667,92.000000,0.0,7466,40.135548,556021,43.227554,1,6922345,81997.0,0.560663,1.342755,False
2021-09-28,286.375000,2021-09-28,293.993333,58.983333,90.233333,0.0,5443,40.472717,472517,41.694517,2,6927788,79168.0,0.572139,1.151916,False


### prepare features

In [6]:
features = df_feat_fr.copy().filter(items=['T_min', 'T_max', 'H_min',
                                           'H_max', 'pos', 'test', 'day_num',
                                          'age_pos', 'age_test'])
features

,T_min,T_max,H_min,H_max,pos,test,day_num,age_pos,age_test
date,,,,,,,,,
2020-05-13,284.926667,290.505000,64.661017,88.135593,881,39013,3,61.104427,55.451567
2020-05-14,285.050000,290.963333,59.406780,84.847458,981,41975,4,60.403670,54.832186
2020-05-15,285.308333,291.920000,57.372881,82.966102,1019,47596,5,60.063788,54.404446
2020-05-16,284.956667,293.500000,53.741379,86.534483,291,16523,6,60.020619,54.480058
2020-05-17,285.598333,294.446667,49.879310,85.500000,141,6611,0,61.248227,58.226895
...,...,...,...,...,...,...,...,...,...
2021-09-26,289.327966,295.911017,63.152542,93.118644,1114,100587,0,41.244165,42.809677
2021-09-27,287.765000,295.293333,59.216667,92.000000,7466,556021,1,40.135548,43.227554
2021-09-28,286.375000,293.993333,58.983333,90.233333,5443,472517,2,40.472717,41.694517


### Prepare dataset

In [7]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)
dataset = (dataset-data_mean)/data_std

In [8]:
dataset.shape[1]

9

In [9]:
PAST_HISTORY

14

In [10]:
dataset.shape[1]

9

### Load model

In [11]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP)

/anaconda3/envs/coronavirusModel/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:1061: UserWarning: tcn.tcn is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


CPU times: user 1.36 s, sys: 63.5 ms, total: 1.43 s
Wall time: 1.69 s


In [12]:
multi_step_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 14, 9)]           0         
_________________________________________________________________
tcn (TCN)                    (None, 64)                43136     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 7)                 455       
Total params: 43,591
Trainable params: 43,591
Non-trainable params: 0
_________________________________________________________________


In [13]:
multi_step_model.inputs[0].dtype

tf.float32

In [14]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_tcn"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: ./keras_tcn/assets


INFO:tensorflow:Assets written to: ./keras_tcn/assets


### Save model TFlite

In [15]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE)

File ./serverless/tensorflow_lite_on_aws_lambda/converted_model_20211004_09_10_10.tflite moved!


In [16]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

In [17]:
open(PATH_MDL_MULTI_TFLITE_FILE, "wb").write(tflite_model)

188752

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [18]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[429 - 443]
[436 - 450]
[443 - 457]
[450 - 464]
[457 - 471]
[464 - 478]
[471 - 485]
[478 - 492]
[485 - 499]
9


array([[10803.118 ,  5189.6836,  8209.056 , 14307.943 , 14013.451 ,
        13030.132 , 13237.821 , 12943.4   ,  6850.291 ,  5076.327 ,
        17426.22  , 14922.551 , 13669.156 , 14062.6   , 13524.233 ,
         7096.1226,  7034.626 , 14198.331 , 18265.172 , 15847.949 ,
        14747.983 , 14318.767 ,  8467.92  ,  1814.8662, 14326.796 ,
        15907.879 , 14164.119 , 13057.618 , 11945.486 ,  7424.291 ,
         3791.7969, 11514.87  , 14324.931 , 12671.196 , 11402.303 ,
        11727.681 ,  7613.127 ,  2506.6797, 11933.139 , 12927.409 ,
        11631.558 , 10958.156 ,  7878.8926,  6275.1147,  3822.1719,
         8826.252 ,  8726.592 ,  7938.5425,  7728.9897,  6350.4204,
         5330.6953,  3950.7734,  7640.366 ,  7253.7183,  6779.8374,
         6546.6284,  5336.7734,  5330.7803,  5111.9395,  5966.573 ,
         6496.1123,  6163.1494,  5762.5864]], dtype=float32)

#### Future days

In [19]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [20]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [21]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE)

In [22]:
PATH_MDL_MULTI_TFLITE_FILE

'./serverless/tensorflow_lite_on_aws_lambda/converted_model.tflite'

### Predict reloaded model

#### Past days

In [23]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [24]:
x_multi.shape

(1, 14, 9)

In [25]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [26]:
dataset.shape

(506, 9)

In [27]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[429 - 443]
[436 - 450]
[443 - 457]
[450 - 464]
[457 - 471]
[464 - 478]
[471 - 485]
[478 - 492]
[485 - 499]


In [28]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arrays : 9 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [29]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [30]:
y_multi_pred

array([[-0.208339  , -0.6097248 , -0.39382613,  0.04227173,  0.02121419,
        -0.04909757, -0.0342468 , -0.05529922, -0.4909839 , -0.6178303 ,
         0.26524252,  0.08621889, -0.00340444,  0.02472854, -0.01376706,
        -0.47340584, -0.47780314,  0.03443396,  0.32523137,  0.15238905,
         0.07373655,  0.04304564, -0.3753162 , -0.8510394 ,  0.04361975,
         0.15667427,  0.03198767, -0.04713213, -0.1266546 , -0.44994032,
        -0.70967996, -0.15744558,  0.04348642, -0.07476306, -0.16549462,
        -0.14222866, -0.4364377 , -0.8015716 , -0.12753749, -0.05644268,
        -0.14910188, -0.19725311, -0.41743428, -0.5321116 , -0.707508  ,
        -0.34969383, -0.35682   , -0.41316903, -0.428153  , -0.5267269 ,
        -0.5996418 , -0.6983124 , -0.43449   , -0.46213704, -0.49602166,
        -0.51269716, -0.5992072 , -0.5996357 , -0.61528385, -0.5541737 ,
        -0.51630926, -0.5401176 , -0.5687597 ]], dtype=float32)

In [31]:
y_multi_pred_out

array([[-0.20833898, -0.60972488, -0.39382607,  0.04227179,  0.02121407,
        -0.04909766, -0.0342468 , -0.05529928, -0.49098393, -0.6178301 ,
         0.26524252,  0.08621895, -0.00340438,  0.0247286 , -0.01376712,
        -0.4734059 , -0.47780299,  0.0344339 ,  0.32523155,  0.15238911,
         0.07373661,  0.0430457 , -0.37531617, -0.85103935,  0.04361987,
         0.15667433,  0.03198773, -0.04713202, -0.12665454, -0.44994029,
        -0.70967972, -0.15744555,  0.04348654, -0.07476294, -0.16549453,
        -0.14222866, -0.43643773, -0.80157161, -0.12753743, -0.05644271,
        -0.14910191, -0.19725311, -0.41743425, -0.53211159, -0.70750809,
        -0.34969375, -0.35681999, -0.413169  , -0.42815298, -0.5267269 ,
        -0.5996418 , -0.6983124 , -0.43448994, -0.46213701, -0.49602163,
        -0.5126971 , -0.59920722, -0.59963572, -0.61528385, -0.55417371,
        -0.51630926, -0.54011762, -0.56875968]])

In [32]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [33]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arrays : 1 / arrays shape: (1, 14, 9)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [34]:
y_future_pred_out

array([[-0.55822915, -0.59914285, -0.62153637, -0.52475232, -0.50238049,
        -0.55130613, -0.59301567]])

In [35]:
y_future_pred

array([[-0.5582291, -0.5991429, -0.6215364, -0.5247523, -0.5023805,
        -0.5513061, -0.5930157]], dtype=float32)

In [36]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda
    
- Execute : sls deploy -v

In [37]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd ./serverless/tensorflow_lite_on_aws_lambda\nserverless deploy -v'

In [38]:
open('deploy_serverless.sh', "w").write(str_exe)
os.chmod('deploy_serverless.sh', stat.S_IRWXU)

In [39]:
!cat ./deploy_serverless.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd ./serverless/tensorflow_lite_on_aws_lambda
serverless deploy -v

In [40]:
!./deploy_serverless.sh

Serverless: Deprecation warning: Resolution of lambda version hashes was improved with better algorithm, which will be used in next major release.
            Switch to it now by setting "provider.lambdaHashingVersion" to "20201221"
            More Info: https://www.serverless.com/framework/docs/deprecations/#LAMBDA_HASHING_VERSION_V2
Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda/.serverless/requirements.txt...
Serverless: Using static cache of requirements found at /Users/gregory/Library/Caches/serverless-python-requirements/eddadf68f9f2c21cf494701082a5393692d7b3c4711137913b3ed1b03bec68cf_slspyc ...
Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: In

### API AWS real Test

#### Past days

In [41]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[429 - 443]
[436 - 450]
[443 - 457]
[450 - 464]
[457 - 471]
[464 - 478]
[471 - 485]
[478 - 492]
[485 - 499]
status code :  200
[[[-0.20833897590637207, -0.6097248196601868, -0.39382606744766235, 0.04227179288864136, 0.02121409773826599, -0.049097657203674316, -0.03424683213233948]], [[-0.05529928207397461, -0.4909839332103729, -0.6178300976753235, 0.2652425765991211, 0.0862189531326294, -0.0034043490886688232, 0.024728596210479736]], [[-0.013767123222351074, -0.4734058976173401, -0.4778030216693878, 0.03443402051925659, 0.32523155212402344, 0.15238898992538452, 0.07373654842376709]], [[0.04304569959640503, -0.37531617283821106, -0.8510393500328064, 0.04361993074417114, 0.156674325466156, 0.03198772668838501, -0.047132015228271484]], [[-0.1266545057296753, -0.4499402642250061, -0.7096797227859497, -0.15744554996490479, 0.04348653554916382, -0.07476294040679932, -0.16549453139305115]], [[-0.14222866296768188, -0.436437726020813, -0.8015716075897217, -0.12753742933273315, -0.0564427077770

In [42]:
len(json_list_list_x)

23443

In [43]:
resp.json()

[[[-0.20833897590637207,
   -0.6097248196601868,
   -0.39382606744766235,
   0.04227179288864136,
   0.02121409773826599,
   -0.049097657203674316,
   -0.03424683213233948]],
 [[-0.05529928207397461,
   -0.4909839332103729,
   -0.6178300976753235,
   0.2652425765991211,
   0.0862189531326294,
   -0.0034043490886688232,
   0.024728596210479736]],
 [[-0.013767123222351074,
   -0.4734058976173401,
   -0.4778030216693878,
   0.03443402051925659,
   0.32523155212402344,
   0.15238898992538452,
   0.07373654842376709]],
 [[0.04304569959640503,
   -0.37531617283821106,
   -0.8510393500328064,
   0.04361993074417114,
   0.156674325466156,
   0.03198772668838501,
   -0.047132015228271484]],
 [[-0.1266545057296753,
   -0.4499402642250061,
   -0.7096797227859497,
   -0.15744554996490479,
   0.04348653554916382,
   -0.07476294040679932,
   -0.16549453139305115]],
 [[-0.14222866296768188,
   -0.436437726020813,
   -0.8015716075897217,
   -0.12753742933273315,
   -0.056442707777023315,
   -0.1491019

In [44]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [45]:
y_multi_pred_out

array([[-0.20833898, -0.60972482, -0.39382607,  0.04227179,  0.0212141 ,
        -0.04909766, -0.03424683, -0.05529928, -0.49098393, -0.6178301 ,
         0.26524258,  0.08621895, -0.00340435,  0.0247286 , -0.01376712,
        -0.4734059 , -0.47780302,  0.03443402,  0.32523155,  0.15238899,
         0.07373655,  0.0430457 , -0.37531617, -0.85103935,  0.04361993,
         0.15667433,  0.03198773, -0.04713202, -0.12665451, -0.44994026,
        -0.70967972, -0.15744555,  0.04348654, -0.07476294, -0.16549453,
        -0.14222866, -0.43643773, -0.80157161, -0.12753743, -0.05644271,
        -0.14910191, -0.19725311, -0.41743425, -0.53211159, -0.70750809,
        -0.34969375, -0.35681999, -0.413169  , -0.42815298, -0.52672684,
        -0.5996418 , -0.6983124 , -0.43448994, -0.46213701, -0.49602163,
        -0.5126971 , -0.59920722, -0.59963572, -0.61528385, -0.55417371,
        -0.51630926, -0.54011762, -0.56875968]])

In [46]:
y_multi_pred

array([[-0.208339  , -0.6097248 , -0.39382613,  0.04227173,  0.02121419,
        -0.04909757, -0.0342468 , -0.05529922, -0.4909839 , -0.6178303 ,
         0.26524252,  0.08621889, -0.00340444,  0.02472854, -0.01376706,
        -0.47340584, -0.47780314,  0.03443396,  0.32523137,  0.15238905,
         0.07373655,  0.04304564, -0.3753162 , -0.8510394 ,  0.04361975,
         0.15667427,  0.03198767, -0.04713213, -0.1266546 , -0.44994032,
        -0.70967996, -0.15744558,  0.04348642, -0.07476306, -0.16549462,
        -0.14222866, -0.4364377 , -0.8015716 , -0.12753749, -0.05644268,
        -0.14910188, -0.19725311, -0.41743428, -0.5321116 , -0.707508  ,
        -0.34969383, -0.35682   , -0.41316903, -0.428153  , -0.5267269 ,
        -0.5996418 , -0.6983124 , -0.43449   , -0.46213704, -0.49602166,
        -0.51269716, -0.5992072 , -0.5996357 , -0.61528385, -0.5541737 ,
        -0.51630926, -0.5401176 , -0.5687597 ]], dtype=float32)

In [47]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [48]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
# REQUEST URL_PREDICT = 'https://yl0910jrga.execute-api.us-east-2.amazonaws.com/dev/infer' 
resp = requests.post(URL_PREDICT, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[-0.5582291483879089, -0.599142849445343, -0.621536374092102, -0.5247523188591003, -0.5023804903030396, -0.5513061285018921, -0.5930156707763672]]]


In [49]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[-0.55822915, -0.59914285, -0.62153637, -0.52475232, -0.50238049,
        -0.55130613, -0.59301567]])

In [50]:
y_future_pred

array([[-0.5582291, -0.5991429, -0.6215364, -0.5247523, -0.5023805,
        -0.5513061, -0.5930157]], dtype=float32)

In [51]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
